In [1]:
import pandas as pd
import pandas as pd
from requests import get
from bs4 import BeautifulSoup

In [2]:
player_data = pd.read_csv("player_team_name_id.csv")
player_data

,team_name,player_name,player_id
0,Afghanistan,Hashmatullah Shahidi,440970
1,Afghanistan,Rahmanullah Gurbaz,974087
2,Afghanistan,Ibrahim Zadran,921509
3,Afghanistan,Reyaz Hassan,1061090
4,Afghanistan,Rahmat Shah Zurmati,533956
...,...,...,...
145,Sri Lanka,Dunith Wellalage,1152427
146,Sri Lanka,Kasun Rajitha,499594
147,Sri Lanka,Matheesha Pathirana,1194795
148,Sri Lanka,Lahiru Kumara,784375


In [3]:
player_data.isnull().sum()

team_name      0
player_name    0
player_id      0
dtype: int64

# Extract Performance Data

1. Batting: `https://stats.espncricinfo.com/ci/engine/player/<:playerID>.html?class=2;template=results;type=batting;view=match`
2. Bowling: `https://stats.espncricinfo.com/ci/engine/player/<:playerID>.html?class=2;template=results;type=bowling;view=match`
3. AllRound: `https://stats.espncricinfo.com/ci/engine/player/<:playerID>.html?class=2;template=results;type=allround;view=match`


## Extracting Player Role

This step is to make the right query and extract the data.

In [23]:
# 253802, Virat Kohli

def find_player_role(player_id):
    try:
        fetch_result = get(f"https://stats.espncricinfo.com/ci/engine/player/{player_id}.html?class=2;type=allround")

        soup = BeautifulSoup(fetch_result.text, 'html.parser')

        # Select first p tag in ciPhotoContainer class
        player_name = soup.select(".ciPhotoContainer p")[0].contents[1].text.strip().replace(" ", "-").lower()

        fetch_result = get(f"https://www.espncricinfo.com/cricketers/{player_name}-{player_id}")
        soup = BeautifulSoup(fetch_result.text, 'html.parser')

        # Find all a tags that point to https://www.espncricinfo.com/cricketers/compare/{player_name}-{player_id}

        player_role = ""
        aList = soup.find_all("a", href=f"/cricketers/compare/{player_name}-{player_id}")

        for a in aList:
            parent_div = a.parent.parent
            player_role = parent_div.contents[2].text.strip()
        
        return player_role
    except:
        return None

player_data["player_role"] = player_data["player_id"].apply(find_player_role)

In [25]:
player_data.isnull().sum()

team_name      0
player_name    0
player_id      0
player_role    1
dtype: int64

In [27]:
# print the player_data["player_role"] that's null

player_data[player_data["player_role"].isnull()]

,team_name,player_name,player_id,player_role
78,Netherlands,Vikram Singh,35281,None


In [37]:
# Set player_role as "Opening Batter" where player_id = 35281

player_data.loc[player_data["player_id"] == 35281, "player_role"] = "Opening Batter"

In [38]:
player_data.isnull().sum()

team_name      0
player_name    0
player_id      0
player_role    0
dtype: int64

In [39]:
player_data["player_role"].value_counts()

Bowler                 50
Allrounder             22
Wicketkeeper Batter    20
Top order Batter       14
Batting Allrounder     11
Opening Batter         10
Bowling Allrounder     10
Batter                  6
Middle order Batter     6
                        1
Name: player_role, dtype: int64

In [ ]:
player_data.to_csv("player_{team_name_id_role}.csv", index=False)